In [ ]:
# deal with valence 2 nodes (intermediaries on one way streets)
# DOES NOT WORK. DO NOT RUN!

# nodes with degree two. i.e. they are in between going from one place to another.
valence_2_nodes = set([k for k, v in g.degree() if v == 2])
print('original len valence_2_nodes: ' + str(len(valence_2_nodes)))

def right_node(node):
#     print('right-node:' + node)
    out_edges = [x for x in g.out_edges(node)]
    
    if len(out_edges) > 0:
        return [x for x in g.out_edges(node)][0][1]
        
    else:
        return ''

def left_node(node):
#     print('left-node:' + node)
    in_edges = [x for x in g.in_edges(node)]
    if len(in_edges) > 0:
        return [x for x in g.in_edges(node)][0][0]
    else:
        return ''


newly_linked = {}

while len(valence_2_nodes) > 0:
    node = list(valence_2_nodes)[0]
    
    old_right, new_right = node, right_node(node)
    old_left, new_left = node, left_node(node)
    
    # edge case with two cyclic nodes.
    # just choose one to win and call it a day.
    if new_left == new_right:
        g.remove_nodes_from([node])
        valence_2_nodes -= set([node, new_left])
        continue
#     print(node)
#     print(new_left)
#     print(new_right)
#     print('---')
        

    nodes_for_deletion = [node]
    distance = 0

    # find leftmost vertex with valence 2
    while (new_left in valence_2_nodes) and (new_left not in nodes_for_deletion) and (new_left in g.nodes()):
        distance += g.get_edge_data(new_left, old_left).get('havlen', 0) # add distance
        old_left = new_left # re-assign the rightmost valence 2 node.
        nodes_for_deletion.insert(0, old_left) # mark the rightmost v2 node for deletion
        new_left = left_node(old_left) # reassign the rightmost node of questionable valence

    # add distance to new leftmost valence != 2 node
    if new_left != '':
        distance += g.get_edge_data(new_left, old_left).get('havlen', 0)

    # find rightmost vertex with valence 2
    while new_right in valence_2_nodes and (new_right not in nodes_for_deletion) and (new_right in g.nodes()):
        distance += g.get_edge_data(old_right, new_right).get('havlen', 0)
        old_right = new_right
        nodes_for_deletion.append(old_right)
        new_right = right_node(old_right)

    # add distance to new rightmost valence != 2 node, if a more rightmost node exists
    if new_right != '':
        distance += g.get_edge_data(old_right, new_right).get('havlen', 0)
       
    # remove old useless nodes
    g.remove_nodes_from(nodes_for_deletion)
    valence_2_nodes -= set(nodes_for_deletion)
    
    # add new edge between end nodes
    g.add_weighted_edges_from([(new_left, new_right, distance)], weight='havlen')
    
    newly_linked[(new_left, new_right)] = [nodes_for_deletion]


print('num_valence_2_nodes remaining: ' + str(len(valence_2_nodes)) + "(should be 0)")
# pprint(newly_linked)

## Remove intermediary two-way nodes

In [1]:
    
def intermediary_two_way_node(node):
    '''returns true if the given node is intermediary on a two way street.
       returns false if given node is not intermediary on a two way street.
       errors if node does not exist.'''
    out_edges = set([item for sublist in g.out_edges(node) for item in sublist]) - set([node])
    in_edges = set([item for sublist in g.in_edges(node) for item in sublist]) - set([node])

    if len(out_edges) == 2 and len(in_edges) == 2:
        return in_edges == out_edges
    else:
        return False

def right_two_way_node(node):
    if intermediary_two_way_node(node):
        return [x for x in g.out_edges(node)][1][1]
    else:
        return False

def left_two_way_node(node):
    if intermediary_two_way_node(node):
        return [x for x in g.out_edges(node)][0][1]
    else:
        return False
    
def compute_distances_of_walk(walk_list):
    '''given a set of walking nodes, Compute the distance between them.'''
    distance = 0
    start_node = walk_list[0]
    
    for next_node in walk_list[1:]:
        sprint = g.get_edge_data(start_node, next_node).get('havlen', 0)
        distance += sprint
        start_node = next_node
    return distance


def walk_right(node):
    if not intermediary_two_way_node(node):
        return {'walked': [node],
                'distance': 0}
    
    # compute node sequence
    walked = [node]
    next_node = right_two_way_node(node)
    
    while intermediary_two_way_node(next_node) and (next_node not in walked):
        walked.append(next_node)
        next_node = right_two_way_node(next_node)
      
    # append non-intermediary node
    walked.append(next_node)
    
    # compute distances    
    return {'walked': walked,
            'distance': compute_distances_of_walk(walked)}

def walk_left(node):
    return {'walked': [node],
            'distance': 0}
    
    walked = [node]
    next_node = left_two_way_node(node)
    
    while intermediary_two_way_node(next_node) and (next_node not in walked):
        walked.append(next_node)
        next_node = left_two_way_node(next_node)
    
    # append non-intermediary node
    walked.append(next_node)
    
    return {'walked': walked,
            'distance': compute_distances_of_walk(walked)}

    

In [ ]:
# get the intermediate nodes
intermediary_2w = set([x for x in g.nodes() if intermediary_two_way_node(x)])

# walk the grah, and remove intermediary nodes
while len(intermediary_2w) > 0:        
    node = list(intermediary_2w)[0]
#     print(node)

        
    right = walk_right(node)
    left = walk_left(node)
    
    # all walked nodes (use a set to eliminate duplicate start node)
    all_walked = set(left['walked'] + right['walked'])
    
    # the total distance walked
    all_distance = left['distance'] + right['distance']
    
    # modify graph
    # Add new edge between non-intermediary nodes
    g.add_weighted_edges_from([(left['walked'][-1], right['walked'][-1], all_distance)], weight='havlen')
    # delete intermediary nodes
    nodes_for_deletion = set(left['walked'][:-1] + right['walked'][:-1])
    g.remove_nodes_from(nodes_for_deletion)
    
    # modify intermediary list
#     print(len(intermediary_2w))
    intermediary_2w -= nodes_for_deletion
    
    # edge case with single node cycle
    if len(nodes_for_deletion) == 0:
        intermediary_2w -= set([node])
        
#     print(len(intermediary_2w)) 
    
    
#     print(node)
#     print(nodes_for_deletion)
#     print(all_walked)
#     print(all_distance)
#     print('---')
    
    

In [ ]:
# cleanup
nodes_to_remove = []
for node in g.nodes():
    if not g.nodes()[node].get('id'):
        nodes_to_remove.append(node)
        
g.remove_nodes_from(nodes_to_remove)

In [ ]:
len(g.nodes())